Open and explore data from:
    https://figshare.com/articles/1000homes/1270900
    
To open Biom-format files:
    $ pip install biom-format
    
For more on Biom-format files:
    http://biom-format.org
    https://github.com/biocore/biom-format/issues/622
    
To do:
    Avoid using append. Try using merge instead to join dataframes
    http://pandas.pydata.org/pandas-docs/stable/merging.html
    http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.07-Merge-and-Join.ipynb

In [1]:
import sys, os
os.chdir('data/otu_tables_wTax')

In [2]:
import pandas as pd
import numpy as np

def exploding_panda(_bt):
    """BIOM->Pandas dataframe converter

    Parameters
    ----------
    _bt : biom.Table
        BIOM table

    Returns
    -------
    pandas.DataFrame
        The BIOM table converted into a DataFrame
        object.

    References
    ----------
    Based on this answer on SO:
    http://stackoverflow.com/a/17819427/379593
    """
    m = _bt.matrix_data
    data = [pd.SparseSeries(m[i].toarray().ravel()) for i in np.arange(m.shape[0])]
    out = pd.SparseDataFrame(data, index=_bt.ids('observation'),
                             columns=_bt.ids('sample'))

    return out

In [3]:
import biom
from biom import parse_table, load_table
fname = 'ITS_otu_table_wTax.biom'
with open(fname) as f: 
    table = parse_table(f)

In [4]:
table

74851 x 2688 <class 'biom.table.Table'> with 4430534 nonzero entries (2% dense)

In [5]:
print table.head()

# Constructed from biom file
#OTU ID	242.O	919.O	330.O	445.O	733.O
OTU_360	30963.0	2.0	1.0	19.0	0.0
OTU_5	2.0	53536.0	6.0	36.0	759.0
OTU_47557	0.0	0.0	1.0	0.0	0.0
OTU_45150	69.0	23.0	202.0	1963.0	9.0
OTU_3	365.0	9387.0	671.0	95.0	29932.0


In [6]:
# print ids along observation axis
print table.ids(axis='observation')

[u'OTU_360' u'OTU_5' u'OTU_47557' ..., u'OTU_55963' u'OTU_47187'
 u'OTU_70298']


In [7]:
# print ids along the sample axis
print table.ids()

[u'242.O' u'919.O' u'330.O' ..., u'1459.O' u'1454.O' u'1267.O']


In [8]:
# to return the index of the identified sample/observation
print table.index('OTU_5','observation')

1


In [9]:
# find out how many nonzero elts there are; returns a numpy array
print table.nonzero_counts('observation',binary=True)

[ 305 2371   32 ...,    1    1    1]


In [10]:
# remove empty samples or observations from the table
table_new = table.remove_empty(axis='whole',inplace=False)
table_new # looks like nothing was removed

74851 x 2688 <class 'biom.table.Table'> with 4430534 nonzero entries (2% dense)

In [11]:
# randomly subsample without replacement
num_sample = 10
samp_table = table.subsample(num_sample,axis='sample',by_id=True)
print samp_table.head()

# Constructed from biom file
#OTU ID	762.O	935.O	956.O	309.I	557.I
OTU_5	1742.0	3.0	1.0	1439.0	20.0
OTU_47557	0.0	0.0	0.0	0.0	0.0
OTU_45150	450.0	1.0	0.0	0.0	252.0
OTU_3	7.0	3.0	9.0	0.0	288.0
OTU_29537	21.0	0.0	2.0	1.0	2.0


In [12]:
# convert matrix data to a Pandas SparseDataFrame, indexed on
# observation IDs, with the column names as the sample IDs
# excludes metadata

# note: if done on the full table, the notebook crashes. 
# try converting the random subsample
df = samp_table.to_dataframe() 
df.head()

,762.O,935.O,956.O,309.I,557.I,318.I,250.I,1438.I,1332.I,1408.O
OTU_5,1742.0,3.0,1.0,1439.0,20.0,140.0,4.0,0.0,75.0,3.0
OTU_47557,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
OTU_45150,450.0,1.0,0.0,0.0,252.0,1.0,0.0,0.0,0.0,0.0
OTU_3,7.0,3.0,9.0,0.0,288.0,10542.0,171.0,7.0,175.0,334.0
OTU_29537,21.0,0.0,2.0,1.0,2.0,600.0,0.0,1.0,24.0,367.0


In [13]:
help(table.iter)

Help on method iter in module biom.table:

iter(self, dense=True, axis='sample') method of biom.table.Table instance
    Yields ``(value, id, metadata)``
    
    
    Parameters
    ----------
    dense : bool, optional
        Defaults to ``True``. If ``False``, yield compressed sparse row or
        compressed sparse columns if `axis` is 'observation' or 'sample',
        respectively.
    axis : {'sample', 'observation'}, optional
        The axis to iterate over.
    
    Returns
    -------
    GeneratorType
        A generator that yields (values, id, metadata)
    
    Examples
    --------
    >>> import numpy as np
    >>> from biom.table import Table
    
    Create a 2x3 BIOM table:
    
    >>> data = np.asarray([[0, 0, 1], [1, 3, 42]])
    >>> table = Table(data, ['O1', 'O2'], ['S1', 'S2', 'Z3'])
    
    Iter over samples and keep those that start with an Z:
    
    >>> [(values, id, metadata)
    ...     for values, id, metadata in table.iter() if id[0]=='Z']
    [(arr

In [14]:
help(table)

Help on Table in module biom.table object:

class Table(__builtin__.object)
 |  The (canonically pronounced 'teh') Table.
 |  
 |  Give in to the power of the Table!
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Equality is determined by the data matrix, metadata, and IDs
 |  
 |  __getitem__(self, args)
 |      Handles row or column slices
 |      
 |      Slicing over an individual axis is supported, but slicing over both
 |      axes at the same time is not supported. Partial slices, such as
 |      `foo[0, 5:10]` are not supported, however full slices are supported,
 |      such as `foo[0, :]`.
 |      
 |      Parameters
 |      ----------
 |      args : tuple or slice
 |          The specific element (by index position) to return or an entire
 |          row or column of the data.
 |      
 |      Returns
 |      -------
 |      float or spmatrix
 |          A float is return if a specific element is specified, otherwise a
 |          spmatrix object represe

In [15]:
otus = [otu for otu in table.ids(axis='observation')]
samples = [samp for samp in table.ids()]

In [16]:
print samples[0:10]

[u'242.O', u'919.O', u'330.O', u'445.O', u'733.O', u'931.O', u'970.O', u'439.O', u'724.O', u'991.O']


In [17]:
filt_table = table.filter(samples[0:10],axis='sample',invert=False,inplace=False)
print filt_table.ids()

[u'242.O' u'919.O' u'330.O' u'445.O' u'733.O' u'931.O' u'970.O' u'439.O'
 u'724.O' u'991.O']


In [18]:
df = filt_table.to_dataframe() 
df.head()

,242.O,919.O,330.O,445.O,733.O,931.O,970.O,439.O,724.O,991.O
OTU_360,30963.0,2.0,1.0,19.0,0.0,0.0,0.0,0.0,0.0,1.0
OTU_5,2.0,53536.0,6.0,36.0,759.0,12.0,13795.0,77.0,25.0,489.0
OTU_47557,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OTU_45150,69.0,23.0,202.0,1963.0,9.0,6.0,0.0,63.0,0.0,1.0
OTU_3,365.0,9387.0,671.0,95.0,29932.0,6.0,183.0,485.0,521.0,6685.0


In [19]:
# note that df.append appends rows, not columns, so lets now filter along the observation
# axis
del filt_table, df
filt_table = table.filter(otus[0:100],axis='observation',invert=False,inplace=False)
df = filt_table.to_dataframe()
df.head()

,242.O,919.O,330.O,445.O,733.O,931.O,970.O,439.O,724.O,991.O,...,1319.O,1220.O,1436.O,1592.O,NTC8b,1328.O,1413.O,1459.O,1454.O,1267.O
OTU_360,30963.0,2.0,1.0,19.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OTU_5,2.0,53536.0,6.0,36.0,759.0,12.0,13795.0,77.0,25.0,489.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OTU_47557,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OTU_45150,69.0,23.0,202.0,1963.0,9.0,6.0,0.0,63.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OTU_3,365.0,9387.0,671.0,95.0,29932.0,6.0,183.0,485.0,521.0,6685.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# now iterate thru, convert to DF and append
num_otus = len(otus)
print num_otus

74851


In [21]:
chunk_size = 1000;
num_iters = 50; #int(round(num_otus/chunk_size)) # 74
print num_iters

50


In [22]:
del filt_table
# make first DF
filt_table = table.filter(otus[0:chunk_size-1],axis='observation',invert=False,inplace=False)
df_all = filt_table.to_dataframe()

In [23]:
print len(df_all.index)

999


In [24]:
curr_start = chunk_size
for i in range(num_iters):
    print curr_start
    filt_table = table.filter(otus[curr_start:curr_start+chunk_size-1],axis='observation',invert=False,inplace=False)
    df = filt_table.to_dataframe()
    df_all = df_all.append(df)
    curr_start = curr_start+chunk_size
    del filt_table, df

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000


In [25]:
print len(df_all.index)

50949


In [31]:
"""
# see also: https://stackoverflow.com/questions/17098654/how-to-store-a-dataframe-using-pandas
# save as csv
#outfile1 = "ITS_otu1.csv"
outfile1 = fname.replace('biom','csv')
df_all.to_csv(outfile1)
del df_all
"""


# save as pickle
outfile1 = 'ITS_otu1.pkl'
df_all.to_pickle(outfile1)
# to read: df = pd.read_pickle(outfile1)

In [32]:
del df_all
# second half
filt_table = table.filter(otus[curr_start:curr_start+chunk_size-1],axis='observation',invert=False,inplace=False)
df_all2 = filt_table.to_dataframe()
del filt_table

In [33]:
num_iters = 23
curr_start = curr_start+chunk_size
for i in range(num_iters):
    print curr_start
    filt_table = table.filter(otus[curr_start:curr_start+chunk_size-1],axis='observation',invert=False,inplace=False)
    df = filt_table.to_dataframe()
    df_all2 = df_all2.append(df)
    curr_start = curr_start+chunk_size
    del filt_table, df

52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000


ValueError: Column length mismatch: 2688 vs. 0

In [ ]:
# now add in remainder
filt_table = table.filter(otus[curr_start:end],axis='observation',invert=False,inplace=False)
df = filt_table.to_dataframe()
del filt_table
df_all2 = df_all2.append(df)

In [34]:
# save as csv 
#fname = 'ITS_otu_table_wTax.biom'
#outfile = fname.replace('biom','csv')
#outfile2 = 'ITS_otu2.csv'
outfile2 = 'ITS_otu2.pkl'
df_all2.to_pickle(outfile2)
del df_all2

In [ ]:
#counts = [table.data(otu,axis='observation',dense=True) for otu in otus]
#print type(counts)

In [ ]:
#print len(counts)

In [ ]:
#counts

In [ ]:
#df = pd.DataFrame(counts[0],columns=['samples']) 

In [ ]:
#part_table = table[0,:] # first row; note: can only slice over 1 axis
#part_table

In [ ]:
#print part_table

In [ ]:
# note: the following line of code causes the notebook to crash if run
# on the whole table; probably uses too much memory

#df2 = exploding_panda(samp_table) 
#df2.head()